## ExLlama Fast Inference
Supports ExLlama WebUI as well as [Oobabooga WebUI API imitation](https://gist.github.com/BlankParenthesis/4f490630b6307ec441364ab64f3ce900)

Up to 34B 4-bit on 2x T4, and 13B 4-bit on 1x P100 or 1x T4
### Installation

In [ ]:
# Kaggle
%cd /kaggle/

# Colab
# %cd /content/

# Install ExLlama and deps
!pip install -q --pre torch --index-url https://download.pytorch.org/whl/nightly/cu118
!pip install -q safetensors sentencepiece ninja
!pip install -q huggingface_hub

!git clone https://github.com/turboderp/exllama
%cd exllama

# Install WebUI deps
!pip install -q flask waitress

# Install deps for Oobabooga WebUI API imitation
!wget "https://gist.githubusercontent.com/BlankParenthesis/4f490630b6307ec441364ab64f3ce900/raw/38f4feb8ea2c023907eaacf4a98c645bca2dfe3a/api.py"
!pip install -q flask_sock

# Install localtunnel to access Flask/API
!npm install localtunnel

### Model download
Download using HuggingFace repo ID

In [ ]:
# Login to hub (to access private models)
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Full repo download method

# Select model
repo_id = "TheBloke/Chronoboros-33B-GPTQ"
#repo_id = "TheBloke/chronos-33b-GPTQ"
#repo_id = "ausboss/llama-30b-supercot-4bit"
#repo_id = "CalderaAI/30B-Lazarus-GPTQ4bit"

#repo_id = "TheBloke/Llama-2-13B-GPTQ"
#repo_id = "TheBloke/chronos-hermes-13B-GPTQ"
#repo_id = "TehVenom/Metharme-13b-4bit-GPTQ"
#repo_id = "TheBloke/Nous-Hermes-13B-GPTQ"

# Select branch
#revision="main"
revision="gptq-4bit-128g-actorder_True"
#revision="gptq-8bit-128g-actorder_True"

# Download model
from huggingface_hub import snapshot_download
snapshot_download(repo_id=repo_id, revision=revision, local_dir=f"./{repo_id.replace('/', '_')}")

import os
os.environ["MODEL_DIR"] = f"{repo_id.replace('/', '_')}"

print(f"Model dir: './{repo_id.replace('/', '_')}'")

In [ ]:
# Old download method - for repos where multiple versions are in the same branch

# Select model
repo_id = "reeducator/bluemoonrp-30b"
model_filename = "bluemoonrp-30b-4bit-128g.safetensors" # From the model repo

# Select branch
revision="main"

# Download model
from huggingface_hub import hf_hub_download
hf_hub_download(repo_id=repo_id, revision=revision, filename="config.json", local_dir=f"./{repo_id.replace('/', '_')}")
hf_hub_download(repo_id=repo_id, revision=revision, filename="tokenizer.model", local_dir=f"./{repo_id.replace('/', '_')}")
hf_hub_download(repo_id=repo_id, revision=revision, filename=model_filename, local_dir=f"./{repo_id.replace('/', '_')}")

import os
os.environ["MODEL_DIR"] = f"{repo_id.replace('/', '_')}"

print(f"Model dir: './{repo_id.replace('/', '_')}'")

In [ ]:
# Full repo download lora

# Select model
repo_id = "Ruaif/Kimiko_13B"

# Select branch
revision="main"

# Download model
from huggingface_hub import snapshot_download
snapshot_download(repo_id=repo_id, revision=revision, local_dir=f"./{repo_id.replace('/', '_')}")

import os
os.environ["LORA_DIR"] = f"{repo_id.replace('/', '_')}"

print(f"Lora dir: './{repo_id.replace('/', '_')}'")

In [ ]:
# Delete downloaded model
!rm -r $MODEL_DIR
!dir

In [ ]:
# Delete downloaded lora
!rm -r $LORA_DIR
!dir

### Run inference
Select either ExLlama WebUI or Oobabooga WebUI API imitation

Arguments:

-d : Path to directory containing config.json, tokenizer.model and * .safetensors **(use $MODEL_DIR)**

--lora : Path to directory containing adapter_config.json and adapter_model.bin **(use $LORA_DIR)**

-gs : Comma-separated list of VRAM (in GB) to use per GPU device for model layers **(recommend 8,11 for 33/34B models on 2x T4, disable on 1x T4 or 1x P100)**

-l : Maximum sequence length **(2048 for llama 1 models and 4096 for llama 2 models, higher for extended context models)**

-a : alpha for context size extension via embedding extension **(leave at 1 for trained context, ~2.5 for 4k context on llama 1 models, 93 for codellama)**

-cpe : Compression factor for positional embeddings **(set to trained value for linear rope scaling models such as superhot or llongma)**

In [ ]:
# ExLlama WebUI
# Access localtunnel page and input IP as password
!curl ipv4.icanhazip.com
!python ./webui/app.py -d $MODEL_DIR --host "127.0.0.1:5000" -gs 8,11 -l 4096 & npx localtunnel --port 5000

In [ ]:
# Oobabooga WebUI API imitation
# Access localtunnel page and input IP as password
# Standard API: https://X/api
# Streaming API: ws://X/api/v1/stream
!curl ipv4.icanhazip.com
!python api.py -d $MODEL_DIR -gs 8,11 -l 4096 & npx localtunnel --port 5000

### Misc tests

In [ ]:
# Benchmarking speeds
!python test_benchmark_inference.py -d $MODEL_DIR -p -gs 8,11 -l 4096

In [ ]:
# Benchmarking perplexity
!python test_benchmark_inference.py -d $MODEL_DIR -ppl -ppl_ds "./datasets/wikitext2_val_sample.jsonl" -gs 8,11 -l 4096